# Convolutional Neural Network

## Import Module

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Check if GPU is activate
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## Preprocessing


### Load cifar100 dataset

In [ ]:
cifar = datasets.cifar100

(train_images, train_labels), (test_images, test_labels) = cifar.load_data()

### Define the class

In [ ]:
class_names = ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 
               'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 
               'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 
               'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 
               'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 
               'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion',
               'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse',
               'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear',
               'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine',
               'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose',
               'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake',
               'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table',
               'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout',
               'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman',
               'worm']

In [ ]:
# View sample data
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[int(train_labels[i])])
plt.show()

## Data exploration

In [ ]:
# Show data shape
train_images.shape

In [ ]:
train_images[0]

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

## Build the model

### Define the model

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(100, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### Fiting model

In [ ]:
model.fit(train_images, train_labels, epochs=5)

### Testing model

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

### Image predict

In [ ]:
predictions = model.predict(test_images)

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[int(true_label)]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 5
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
plt.tight_layout()
plt.show()

## Try your own

In [ ]:
!wget 'https://www.dw.com/image/48396304_403.jpg'

In [ ]:
from PIL import Image, ImageOps

img = Image.open('/content/48396304_403.jpg')
img

In [ ]:
new_width  = 32
new_height = 32
img = img.resize((new_width, new_height), Image.ANTIALIAS)

In [ ]:
plt.imshow(img)

In [ ]:
img = np.array(img)
img = img / 255

In [ ]:
predictions_array = model.predict(img.reshape(1, 32, 32, 3))
plt.grid(False)
plt.xticks(range(100))
plt.yticks([])
thisplot = plt.bar(range(100), predictions_array[0], color="#777777")
plt.ylim([0, 1])
predicted_label = np.argmax(predictions_array)
plt.show()
class_names[predicted_label]

In [ ]:
def predict(img):
  new_width  = 32
  new_height = 32
  img = np.array(ImageOps.invert(img.resize((new_width, new_height), Image.ANTIALIAS)))
  img = img / 255
  plt.figure(figsize=(6,3))
  plt.subplot(1,2,1)
  plt.imshow(img, cmap=plt.cm.binary)
  plt.subplot(1,2,2)
  predictions_array = model.predict(img.reshape(1, 32, 32, 3))
  plt.grid(False)
  plt.xticks(range(100))
  plt.yticks([])
  thisplot = plt.bar(range(100), predictions_array[0], color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)
  plt.show()
  return class_names[np.argmax(model.predict(img.reshape(1, 32, 32, 3)))]

In [ ]:
predict(img)